## Formalism

### Problem outline

Goal: predict a posterior distribution for the total power output for each sector of Sweden, given data for the features of each turbine $\boldsymbol\theta_{T}$ (type, maximum power, height-above-sea-level of the land they are on, etc), and also a probabilistic forecast for the weather $\boldsymbol\theta_{w}$ (wind speed, temperature, humidity, wind gust speed etc) at each location over the map. 

Let's call this probabilistic weather forecast $P(\boldsymbol\theta_{w} | \mathbf{d}_{w})$ where $\mathbf{d}_{w}$ is all of the "weather data" that went into the Swedish weather models.

Our posterior for the wind power $w_{P,i}$ of a given turbine $i$ given the data is hence given by:

$$P(w_{P,i} | \boldsymbol\theta_{T,i}, \mathbf{d}_{w}) = \int P(w_{P,i} | \boldsymbol\theta_{T,i}, \boldsymbol\theta_{w,i})P(\boldsymbol\theta_{w,i} | \mathbf{d}_{w}) d\boldsymbol\theta_{w,i}$$

ie., the posterior for the wind power given weather and turbine features, integrated over our uncertainty about the weather at that location. Note if we were uncertain about the features of the turbine (eg., some features were missing) we can stick in an integral over the features too to account for uncertainties there.

Now, we don't actually have a distribution for the weather forecast but rather just a little ensemble of ten weather predictions $\{\boldsymbol\theta_{w,i}\}$, which we will assume for now are effectively "draws" from the weather prediction distribution. So, we can approximate the integral above as a sum over this ensemble of weather predictions:

\begin{align}
P(w_{P} | \boldsymbol\theta_{T}, \mathbf{d}_{w}) \simeq \frac{1}{10}\sum_{j=1}^{10} P(w_{P} | \boldsymbol\theta_{T}, \boldsymbol\theta^j_{w})
\end{align}

The thing we actually need to predict is a posterior distribution for the total power output of a given sector (given all of the data about the turbine features and weather). i.e, total output for a sector is defined to be the sum over turbines in that sector:

$$W_{P, S} = \sum_{i\;in\;sector\;S} w_{P,i}$$

and we ultimately want to infer a posterior distribution for that:

$$P(W_{P, S} | \boldsymbol\theta_T, \mathbf{d}_w).$$

Because $W_{P, S}$ is a sum over individual turbines, and the probability distribution of a sum of random variables is the convolution of their individual probability distributions, we have:

$$P(W_{P, S} | \boldsymbol\theta_T, \mathbf{d}_w) = P(w_{P,1} | \boldsymbol\theta_{T,1}, \mathbf{d}_{w}) * P(w_{P,2} | \boldsymbol\theta_{T,2}, \mathbf{d}_{w}) * \dots * P(w_{P,n} | \boldsymbol\theta_{T,n}, \mathbf{d}_{w})$$

over all turbines 1-n in sector S.

So, at the very core of this, the thing we need to model is:

$$P(w_{P} | \boldsymbol\theta_{T}, \boldsymbol\theta_{w}),$$

because from that we can average over the ensemble of weather forecasts, and then convolve together each individual turbine to get the posterior for the total output from each sector (the target for prediction).

### Suggested initial model

I suggest we model $P(w_{P} | \boldsymbol\theta_{T}, \boldsymbol\theta_{w})$ as a neural density estimator. For the first step let's assume that it is Gaussian, ie, 

$$P(w_{P} | \boldsymbol\theta_{T}, \boldsymbol\theta_{w}) = \mathcal{N}(w_{P} | \mu = \mu(\boldsymbol\theta_{T}, \boldsymbol\theta_{w}), \sigma = \sigma(\boldsymbol\theta_{T}, \boldsymbol\theta_{w}))$$

So, the mean and std-dev of the Gaussian are some unknown functions of the inputs (turbine features and weather) that we want to learn.

Let's use a neural network to parameterize the mean and std-dev of that Gaussian as a function of the inputs, ie., we have a neural network that takes inputs 

$$\mathrm{inputs} \; \mathbf{x} = (\boldsymbol\theta_{T}, \boldsymbol\theta_{w})$$

and outputs

$$\mathrm{outputs} \; \mathbf{y} = (\mu, \mathrm{ln}\,\sigma)$$

Then $P(W_{P, S} | \boldsymbol\theta_T, \mathbf{d}_w)$ (for a given time) is just a (complicated) function of $P(w_{P} | \boldsymbol\theta_{T}, \boldsymbol\theta_{w})$, averaged over weather predictions and convolved together over the locations of each turbine in the sector.

Our loss function (to begin with) can just be (negative) log probability for all the total wind power outputs predicted by our model

$$\mathcal{L} = -\sum_{\mathrm{time}\,t, \mathrm{sector}\,S} \mathrm{ln}\,P(W^t_{P,S} | \boldsymbol\theta_T, \mathbf{d}^t_w)$$